**数据集下载**

In [ ]:
!wget https://digix-algo-challenge.obs.cn-east-2.myhuaweicloud.com/2020/cv/6rKDTsB6sX8A1O2DA2IAq7TgHPdSPxJF/train_data.zip

In [ ]:
!unzip train_data.zip

**文件头加载库**

In [ ]:
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.utils import np_utils
from keras.optimizers import Adam
import numpy as np
import cv2
from keras import backend
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization
from keras.layers import *
from tensorflow.python.keras.utils import conv_utils
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions

**读取label.txt，使其转换为数据集**

*参考CSDN博主「Bubbliiiing」的原创文章*
*原文链接：https://blog.csdn.net/weixin_44791964/java/article/details/102779878*

In [ ]:
def generate_arrays_from_file(label_file,batch_size):
    n = len(label_file)
    i = 0
    while True:
      X_train = []
      Y_train = []
      for b in range(batch_size):
        if i==0:
          #内部打乱
          np.random.shuffle(label_file)
        name = label_file[i].split(',')[0]
        # 从文件中读取图像
        img = image.load_img("train_data/" + name, target_size=(224,224))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)
        X_train.append(img)
        Y_train.append(label_file[i].split(',')[1])
        # 读完一个周期后重新开始
        i = (i+1) % n
      X_train = np.array(X_train)
      X_train = X_train.reshape(-1,224,224,3)
      Y_train = np_utils.to_categorical(np.array(Y_train),num_classes=3097)  
      yield (X_train, Y_train)

**数据集训练集验证集划分**

In [ ]:
with open("train_data/label.txt","r") as f:
  items = f.readlines()
#外部打乱
np.random.seed(10101)
np.random.shuffle(items)
#train_test_split
num_val = int(len(items)*0.1)
num_train = len(items) - num_val

**使用ResNet50模型，并在此上进行模型微调**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 首先构建不带分类器的预训练模型
base_model = ResNet50(weights='imagenet', include_top=False)
# 添加全局平均池化层
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
# 添加一个全连接层
x = Dense(4096, activation='relu')(x)
x = BatchNormalization()(x)
# 添加一个分类器，我们有3097个类
predictions = Dense(3097, activation='softmax')(x)
# 构建我们需要训练的完整模型
model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
for layer in model.layers[:143]:
   layer.trainable = False
for layer in model.layers[143:]:
   layer.trainable = True

In [ ]:
# 编译模型（根据官方文档，一定要在锁层以后操作）
#优化器的选择等参数暂不调整 optimizer = Adam(lr=1e-3),metrics = ['accuracy']
model.compile(optimizer= Adam(lr=1e-4), loss='categorical_crossentropy')

In [ ]:
# 在新的数据集上训练几代
checkpoint_period1 = ModelCheckpoint(
                    "./logs1/" + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
                    monitor='acc', 
                    save_weights_only=False, 
                    save_best_only=True, 
                    period=3
                   )
# 学习率下降的方式，acc三次不下降就下降学习率继续训练
reduce_lr = ReduceLROnPlateau(
                monitor='acc', 
                factor=0.5, 
                patience=3, 
                verbose=1
               )

In [ ]:
#batch_size调参参数
batch_size = 64

**训练模型**

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
model.fit(generate_arrays_from_file(items[:num_train],batch_size),
          steps_per_epoch=max(1, num_train//batch_size),
          validation_data=generate_arrays_from_file(items[num_train:],batch_size),
          validation_steps=max(1, num_val//batch_size),
          epochs=1,
          initial_epoch=0,
          callbacks=[checkpoint_period1, reduce_lr])

In [ ]:
for layer in model.layers[:100]:
   layer.trainable = False
for layer in model.layers[100:]:
   layer.trainable = True

In [ ]:
model.compile(optimizer= Adam(lr=1e-4), loss='categorical_crossentropy')

In [ ]:
#from keras.optimizers import SGD
#model.compile(optimizer=SGD(lr=0.0003, momentum=0.9), loss='categorical_crossentropy')

In [ ]:
model.fit(generate_arrays_from_file(items[:num_train],batch_size),
          steps_per_epoch=max(1, num_train//batch_size),
          validation_data=generate_arrays_from_file(items[num_train:],batch_size),
          validation_steps=max(1, num_val//batch_size),
          epochs=4,
          initial_epoch=0,
          callbacks=[checkpoint_period1, reduce_lr])

In [ ]:
model.save('/content/drive/Shared drives/my_new_pan/5_resnet50_finetuned-4096-bn.h5')

In [ ]:
model.fit(generate_arrays_from_file(items[:num_train],batch_size),
          steps_per_epoch=max(1, num_train//batch_size),
          validation_data=generate_arrays_from_file(items[num_train:],batch_size),
          validation_steps=max(1, num_val//batch_size),
          epochs=5,
          initial_epoch=0,
          callbacks=[checkpoint_period1, reduce_lr])

In [ ]:
model.save('/content/drive/Shared drives/my_new_pan/10_resnet50_finetuned-4096-bn.h5')

In [ ]:
#model.load_weights('/content/drive/Shared drives/my_new_pan/10_resnet50_finetuned5.h5')